In [15]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder
import folium
from geopy.geocoders import Nominatim # convert an address to langtitude and longitude
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url = "https://vi.wikipedia.org/wiki/Th%C3%A0nh_ph%E1%BB%91_H%E1%BB%93_Ch%C3%AD_Minh"

r  = requests.get(url)

data = r.text

soup = BeautifulSoup(data,'lxml')

In [3]:
df = []
for i in soup.find_all("table",class_="wikitable sortable")[0].find_all("tr"):
    ls = []
    for j in i.find_all("td"):
        ls.append(j.get_text())
    df.append(ls)
df = pd.DataFrame(df)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,\n\n\nTên\n\nDiện tích (km²)\nDân số (người)\n...,Quận (19)\n,Quận 1\n,"7,72",142.000,10 phường\n,Quận 2\n,"49,79",180.000,11 phường\n,...,"1 thị trấn, 20 xã\n",Hóc Môn\n,"109,17",542.000,"1 thị trấn, 11 xã\n",Nhà Bè\n,"100,43",206.000,"1 thị trấn, 6 xã\n",\n
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Quận (19)\n,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Quận 1\n,"7,72",142.000,10 phường\n,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,Quận 2\n,"49,79",180.000,11 phường\n,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Quận 3\n,"4,92",190.000,14 phường\n,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,Quận 4\n,"4,18",175.000,15 phường\n,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Quận 5\n,"4,27",159.000,15 phường\n,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Quận 6\n,"7,14",233.000,14 phường\n,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [5]:
df.drop(labels=[0,1,2,3,16,24],inplace=True)
df = df.iloc[:,0:3]
df.columns=['District Name','Acreage (Km2)','Population']
df.reset_index(drop=True,inplace=True)
# Remove EOL "\n" String in dataset
for i in df.columns:
    df[i] = df[i].str.replace("\n","",regex=True)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
df['Acreage (Km2)'] = df['Acreage (Km2)'].str.replace(",",".",regex=True)
df['Population'] = df['Population'].str.replace(".","",regex=True)
df['Acreage (Km2)'].astype(float)
df['Population'].astype(int)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df.head()

,District Name,Acreage (Km2),Population
0,Quận 1,7.72,142000
1,Quận 2,49.79,180000
2,Quận 3,4.92,190000
3,Quận 4,4.18,175000
4,Quận 5,4.27,159000


In [12]:
df.shape

(24, 3)

In [13]:
df_hcmc = df.copy()

### Prepare for fourquare dataframe

In [16]:
CLIENT_ID = 'TYNGSEF0SNXTWZU5JEPMP3Z3BSCBA2TQEK2ODCGA114NG4GG' # your Foursquare ID
CLIENT_SECRET = 'G21UECLTFUK3MLIFP4SQK03OR1JMMDI4EPQS4J12TF0CFMP3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TYNGSEF0SNXTWZU5JEPMP3Z3BSCBA2TQEK2ODCGA114NG4GG
CLIENT_SECRET:G21UECLTFUK3MLIFP4SQK03OR1JMMDI4EPQS4J12TF0CFMP3


In [40]:
df_hcm_cor = pd.read_csv("HCM district coordinates.csv")

In [41]:
df_hcm_cor.head()

,District Name,Latitude,Longitude
0,Quận 1,10.777369,106.696646
1,Quận 2,10.782377,106.754713
2,Quận 3,10.780681,106.680866
3,Quận 4,10.758388,106.702021
4,Quận 5,10.755418,106.667333


In [45]:
df_hcm_cor = df_hcm_cor.sort_values(by="District Name",ascending=True)
df_hcmc = df_hcmc.sort_values(by="District Name",ascending=True)

In [46]:
df_hcm_cor['District Name'] == df_hcmc['District Name']

19    True
13    True
12    True
20    True
21    True
14    True
22    True
23    True
15    True
0     True
9     True
10    True
11    True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
18    True
16    True
17    True
Name: District Name, dtype: bool

In [50]:
df_hcmc = df_hcmc.merge(df_hcm_cor.iloc[:,1:3],left_index=True,right_index=True)

In [51]:
df_hcmc

,District Name,Acreage (Km2),Population,Latitude,Longitude
19,Bình Chánh,252.56,705000,10.724583,106.575197
13,Bình Thạnh,20.78,499000,10.812639,106.714579
12,Bình Tân,52.02,784000,10.770324,106.599978
20,Cần Giờ,704.45,71000,10.535622,106.854503
21,Củ Chi,434.77,462000,11.008502,106.518123
14,Gò Vấp,19.73,676000,10.837427,106.666492
22,Hóc Môn,109.17,542000,10.891521,106.600013
23,Nhà Bè,100.43,206000,10.650182,106.729357
15,Phú Nhuận,4.88,163000,10.799939,106.677777
0,Quận 1,7.72,142000,10.777369,106.696646


In [52]:
address = 'Ho Chi Minh'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ho Chi Minh City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ho Chi Minh City are 10.7758439, 106.7017555.


In [56]:
m = folium.Map(location=[latitude, longitude],zoom_start=12)
#url = 'https://cocl.us/sanfran_geojson'
#geo_data_input = f'{url}'
for index,values in df_hcmc.iterrows():
    folium.CircleMarker(
        [values['Latitude'], values['Longitude']],
        radius=2,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(m)  
m